<a href="https://colab.research.google.com/github/Jamnic98/blue-cheese-classifier/blob/main/blue_cheese_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')

!ls


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [77]:
import os
import numpy as np
import shutil
import random

class_names = ['blueCheese', 'notBlueCheese']
root_dir = '/content/drive/MyDrive/'
val_ratio = 0.15
test_ratio = 0.05

all_cheese_image_dirs = {class_names[0]: [], class_names[1]: []}

for class_name in class_names:
  train_URI = root_dir +'train/' + class_name
  if not os.path.isdir(train_URI):
    os.makedirs(train_URI)
  
  test_URI = root_dir +'test/' + class_name
  if not os.path.isdir(test_URI):
    os.makedirs(test_URI)
  
  val_URI = root_dir +'val/' + class_name
  if not os.path.isdir(val_URI):
    os.makedirs(val_URI)

  cheeses = os.listdir(root_dir + class_name)

  for cheese_name in cheeses:
    cheese_dir = root_dir + class_name + '/' + cheese_name

    all_cheese_image_dirs[class_name].extend(
      [cheese_dir + '/'+ name for name in os.listdir(cheese_dir)]
      )  

FileExistsError: ignored

In [75]:
for class_name in class_names:
  cheese_image_dirs = all_cheese_image_dirs[class_name]
  np.random.shuffle(cheese_image_dirs)
  train_FileNames, val_FileNames, test_FileNames = np.split(
    np.array(cheese_image_dirs),
    [int(len(cheese_image_dirs)* (1 - (val_ratio + test_ratio))), 
    int(len(cheese_image_dirs)* (1 - test_ratio))]
  )

  print(train_FileNames.tolist())

  # # Copy-pasting images
  # for train_FileName in train_FileNames.tolist():
  #   shutil.copy(train_FileName, root_dir +'train/' + class_name)

  # for val_FileName in val_FileNames.tolist():
  #   shutil.copy(val_FileName, root_dir +'val/' + class_name)

  # for test_FileName in test_FileNames.tolist():
  #   shutil.copy(test_FileName, root_dir +'test/' + class_name)

['/content/drive/MyDrive/blueCheese/stichelton/stichelton62.jpg', '/content/drive/MyDrive/blueCheese/stichelton/stichelton107.jpg', '/content/drive/MyDrive/blueCheese/bleu des causses/bleu_des_causses34.jpg', '/content/drive/MyDrive/blueCheese/stilton/stilton19.jpg', '/content/drive/MyDrive/blueCheese/roquefort/roquefort104.jpg', '/content/drive/MyDrive/blueCheese/misc/misc22.jpg', '/content/drive/MyDrive/blueCheese/stichelton/stichelton2.jpg', '/content/drive/MyDrive/blueCheese/shropshire blue/shropshire_blue88.jpg', '/content/drive/MyDrive/blueCheese/bleu des causses/bleu_des_causses92.jpg', '/content/drive/MyDrive/blueCheese/stilton/stilton152.jpg', '/content/drive/MyDrive/blueCheese/shropshire blue/shropshire_blue112.jpg', '/content/drive/MyDrive/blueCheese/oxford blue/oxford_blue5.jpg', '/content/drive/MyDrive/blueCheese/misc/misc94.jpg', '/content/drive/MyDrive/blueCheese/bleu des causses/bleu_des_causses24.jpg', '/content/drive/MyDrive/blueCheese/stichelton/stichelton11.jpg', '/